<a href="https://colab.research.google.com/github/AyerDaniel/ds677/blob/fahriye/dl_677_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/Franck-Dernoncourt/pubmed-rct

Cloning into 'pubmed-rct'...
remote: Enumerating objects: 39, done.
remote: Counting objects: 100% (14/14), done.
remote: Compressing objects: 100% (9/9), done.
remote: Total 39 (delta 8), reused 5 (delta 5), pack-reused 25 (from 1)
Receiving objects: 100% (39/39), 177.08 MiB | 17.10 MiB/s, done.
Resolving deltas: 100% (15/15), done.


In [2]:
data_directory = "./pubmed-rct/PubMed_200k_RCT_numbers_replaced_with_at_sign/"

In [3]:
!ls pubmed-rct/PubMed_200k_RCT_numbers_replaced_with_at_sign

dev.txt  test.txt  train.zip


In [4]:
### Getting Train data

from pathlib import Path

zip_file_path = Path("/content/pubmed-rct/PubMed_200k_RCT_numbers_replaced_with_at_sign/train.zip")

if not zip_file_path.exists():
    print(f"Error: {zip_file_path} does not exist!")
else:
    print(f"File found: {zip_file_path}")

File found: /content/pubmed-rct/PubMed_200k_RCT_numbers_replaced_with_at_sign/train.zip


In [5]:
import os

if os.path.exists(zip_file_path):
    print(f"✅ File exists: {zip_file_path}")
    print(f"📏 File size: {os.path.getsize(zip_file_path)} bytes")
else:
    print("❌ File does not exist!")


✅ File exists: /content/pubmed-rct/PubMed_200k_RCT_numbers_replaced_with_at_sign/train.zip
📏 File size: 100175842 bytes


In [6]:
data_path = Path("data/")
train_path = data_path / "train"

if train_path.is_dir():
  print (f"{train_path} exists")
else:
  print(f"creating {train_path}")
  train_path.mkdir(parents=True, exist_ok=True)

creating data/train


In [7]:
import zipfile


with zipfile.ZipFile(zip_file_path, "r") as zip_ref:
    zip_ref.extractall(train_path)


In [8]:
train_file = train_path / "train.txt"

In [9]:
with open(train_file) as f:
  print(f.readlines()[:20])

['###24491034\n', 'BACKGROUND\tThe emergence of HIV as a chronic condition means that people living with HIV are required to take more responsibility for the self-management of their condition , including making physical , emotional and social adjustments .\n', 'BACKGROUND\tThis paper describes the design and evaluation of Positive Outlook , an online program aiming to enhance the self-management skills of gay men living with HIV .\n', 'METHODS\tThis study is designed as a randomised controlled trial in which men living with HIV in Australia will be assigned to either an intervention group or usual care control group .\n', "METHODS\tThe intervention group will participate in the online group program ` Positive Outlook ' .\n", 'METHODS\tThe program is based on self-efficacy theory and uses a self-management approach to enhance skills , confidence and abilities to manage the psychosocial issues associated with HIV in daily life .\n', 'METHODS\tParticipants will access the program for a m

In [10]:
## Getting text data
with open(data_directory + "test.txt") as f:
  print(f.readlines()[:20])

['###24562799\n', 'BACKGROUND\tMany pathogenic processes and diseases are the result of an erroneous activation of the complement cascade and a number of inhibitors of complement have thus been examined for anti-inflammatory actions .\n', 'BACKGROUND\tIt was recently demonstrated that supraphysiological concentrations of the endogenous complement inhibitor MAp@ ( also denoted MAP@ ) protect against myocardial reperfusion injury .\n', 'BACKGROUND\tIn the present study , we examined the association between outcome after acute myocardial infarction ( MI ) and the plasma levels of MAp@ and its related proteins MASP-@ and MASP-@ in patients with first-time MI .\n', 'BACKGROUND\tIn addition , we compared plasma levels of MAp@ , MASP-@ , and MASP-@ in MI patients to levels in a healthy control group .\n', 'METHODS\tA total of @ MI patients and @ control persons were included .\n', 'METHODS\tPlasma samples were obtained and analysed with time-resolved immunofluorometric assays determining the 

In [11]:

def preprocess_data(data_directory):
  input_lines = open(data_directory).readlines()
  abstract_lines = ""
  abstract_samples = []

  for line in input_lines:
    if line.startswith("###"):
      abstract_id = line
      abstract_lines = ""
    elif line.isspace():
      abstract_line_split = abstract_lines.splitlines()

      for abstract_line_number, abstract_line in enumerate(abstract_line_split):
        line_data = {}
        target_text_split = abstract_line.split("\t")
        line_data["target"] = target_text_split[0]
        line_data["text"] = target_text_split[1].lower()
        line_data["line_number"] = abstract_line_number
        line_data["total_lines"] = len(abstract_line_split) - 1
        abstract_samples.append(line_data)
    else:
      abstract_lines += line
  return abstract_samples



In [12]:
data_directory

'./pubmed-rct/PubMed_200k_RCT_numbers_replaced_with_at_sign/'

In [13]:
data_test = "./data/train/"

In [14]:

train_dir = data_test + "train.txt"
val_dir = data_directory + "dev.txt"
test_dir = data_directory + "test.txt"

train_dir, val_dir, test_dir

('./data/train/train.txt',
 './pubmed-rct/PubMed_200k_RCT_numbers_replaced_with_at_sign/dev.txt',
 './pubmed-rct/PubMed_200k_RCT_numbers_replaced_with_at_sign/test.txt')

In [15]:
train_data_all = preprocess_data(train_dir)
print(len(train_data_all))

2211861


In [16]:
import pandas as pd

train_data_all = pd.DataFrame(train_data_all)

In [17]:
train_data_all.head()

,target,text,line_number,total_lines
0,BACKGROUND,the emergence of hiv as a chronic condition me...,0,10
1,BACKGROUND,this paper describes the design and evaluation...,1,10
2,METHODS,this study is designed as a randomised control...,2,10
3,METHODS,the intervention group will participate in the...,3,10
4,METHODS,the program is based on self-efficacy theory a...,4,10


In [18]:
train_data_all.shape[0]

2211861

In [19]:
val_data_all = preprocess_data(val_dir)
val_data_all = pd.DataFrame(val_data_all)

test_data_all = preprocess_data(test_dir)
test_data_all = pd.DataFrame(test_data_all)

len(val_data_all), len(test_data_all)

(28932, 29493)

In [20]:
## Data Visualization - Train-Val-Test Data Size

import plotly.graph_objects as go

fig = go.Figure([go.Pie(labels=['Train Size','Val Size', 'Test Size'], values=[train_data_all.shape[0], val_data_all.shape[0], test_data_all.shape[0]])])
fig.update_layout(title='Train,Validation and Test Size')
fig.show()

In [21]:
## Data Visualization - Percentage of Class

fig = go.Figure([go.Pie(labels=['OBJECTIVE', 'METHODS', 'RESULTS', 'CONCLUSIONS', 'BACKGROUND'], values=train_data_all.target.value_counts())])
fig.update_layout(title='Percentage of Class (OBJECTIVE, METHODS, RESULTS, CONCLUSIONS, BACKGROUND)')
fig.show()


In [22]:
## changing the train-test-val data to numeric values

from sklearn.preprocessing import OneHotEncoder

one_hot_encoder = OneHotEncoder(sparse_output=False)

train_one_hot = one_hot_encoder.fit_transform(train_data_all[["target"]])
val_one_hot = one_hot_encoder.transform(val_data_all[["target"]])
test_one_hot = one_hot_encoder.transform(test_data_all[["target"]])

train_one_hot

array([[1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0.],
       ...,
       [0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1.]])

In [23]:
## Label encoder
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
train_labels = label_encoder.fit_transform(train_data_all["target"])
val_labels = label_encoder.transform(val_data_all["target"])
test_labels = label_encoder.transform(test_data_all["target"])

train_labels

array([0, 0, 2, ..., 4, 4, 4])

In [24]:
label_encoder.classes_


array(['BACKGROUND', 'CONCLUSIONS', 'METHODS', 'OBJECTIVE', 'RESULTS'],
      dtype=object)

In [26]:
import torch
from torch import nn
from transformers import DistilBertModel

class TransformerBlock(nn.Module):
    def __init__(self, hidden_size, num_heads):
        super().__init__()
        self.attention = nn.MultiheadAttention(hidden_size, num_heads, batch_first=True)
        self.norm1 = nn.LayerNorm(hidden_size)
        self.ffn = nn.Sequential(
            nn.Linear(hidden_size, hidden_size * 4),
            nn.GELU(),
            nn.Linear(hidden_size * 4, hidden_size)
        )
        self.norm2 = nn.LayerNorm(hidden_size)

    def forward(self, x, attn_mask=None):
        attn_output, _ = self.attention(x, x, x, attn_mask=attn_mask)
        x = self.norm1(x + attn_output)
        ffn_output = self.ffn(x)
        x = self.norm2(x + ffn_output)
        return x


In [31]:
## DistilBert model

import torch
from torch import nn
from transformers import DistilBertModel

class DistilBERT(nn.Module):
    def __init__(self,
                 vocab_size=30522,
                 max_position_embeddings=512,
                 hidden_size=768,
                 num_layers=6,
                 num_heads=12,
                 num_classes=5,
                ):
        super().__init__()

        self.embeddings = nn.Embedding(vocab_size, hidden_size)
        self.position_embeddings = nn.Embedding(max_position_embeddings, hidden_size)

        self.transformer_blocks = nn.ModuleList([
            TransformerBlock(hidden_size, num_heads) for _ in range(num_layers)
        ])

        self.classifier = nn.Linear(hidden_size, num_classes)

    def forward(self, input_ids):
        batch_size, seq_len = input_ids.shape
        positions = torch.arange(0, seq_len, device=input_ids.device).unsqueeze(0)

        x = self.embeddings(input_ids) + self.position_embeddings(positions)

        for layer in self.transformer_blocks:
            x = layer(x)

        cls_token = x[:, 0]  # First token
        return self.classifier(cls_token)



In [28]:
## DistilBert model with Dropout

import torch
from torch import nn

class TransformerBlockDropout(nn.Module):
    def __init__(self, hidden_size, num_heads, dropout=0.1):
        super().__init__()
        self.attention = nn.MultiheadAttention(hidden_size, num_heads, dropout=dropout, batch_first=True)
        self.norm1 = nn.LayerNorm(hidden_size)
        self.ffn = nn.Sequential(
            nn.Linear(hidden_size, hidden_size * 4),
            nn.GELU(),
            nn.Linear(hidden_size * 4, hidden_size)
        )
        self.norm2 = nn.LayerNorm(hidden_size)

    def forward(self, x, attn_mask=None):
        attn_output, _ = self.attention(x, x, x, attn_mask=attn_mask)
        x = self.norm1(x + attn_output)
        ffn_output = self.ffn(x)
        x = self.norm2(x + ffn_output)
        return x


In [29]:
## DistilBert model with Dropout

class DistilBERTDropout(nn.Module):
    def __init__(self,
                 vocab_size=30522,
                 max_position_embeddings=512,
                 hidden_size=768,
                 num_layers=6,
                 num_heads=12,
                 num_classes=3,
                 dropout=0.1):
        super().__init__()

        self.embeddings = nn.Embedding(vocab_size, hidden_size)
        self.position_embeddings = nn.Embedding(max_position_embeddings, hidden_size)

        self.transformer_blocks = nn.ModuleList([
            TransformerBlockDropout(hidden_size, num_heads, dropout) for _ in range(num_layers)
        ])

        self.classifier = nn.Linear(hidden_size, num_classes)

    def forward(self, input_ids):
        batch_size, seq_len = input_ids.shape
        positions = torch.arange(0, seq_len, device=input_ids.device).unsqueeze(0)

        x = self.embeddings(input_ids) + self.position_embeddings(positions)

        for layer in self.transformer_blocks:
            x = layer(x)

        cls_token = x[:, 0]  # First token
        return self.classifier(cls_token)


In [30]:
from transformers import DistilBertTokenizerFast
tokenizer = DistilBertTokenizerFast.from_pretrained("distilbert-base-uncased")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning:


The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.



tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]